In [1]:
import tensorflow as tf
import pandas as pd
import json
import configparser
from pathlib import Path
import os
    
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling1D

configs = configparser.ConfigParser()
configs.read("configuracoes.conf",encoding='utf-8')

data_folder = Path(configs["DEFAULT"]["folder_path"])
path = data_folder / configs["DEFAULT"]["file"]
objeto = configs["DEFAULT"]["objeto"]
regiao = configs["DEFAULT"]["regiao"]
look_back = 12 #Quantidade de meses anteriores usado para prever
forecast_horizon = int(configs["DEFAULT"]["meses"]) #Quantidade de meses previstos

# Prever Cesta

In [2]:
#Carrega os dados
cesta = pd.read_excel(path)
cesta.drop(['ano'], axis=1, inplace=True)
cesta = cesta / 1000
cesta = cesta.iloc[::, ::]

#Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
for n_step in range(1, look_back + forecast_horizon):
    cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
cesta.dropna(inplace=True)
cesta.reset_index(drop=True, inplace=True)

#Divide os dados em Treino e Teste
X_train = cesta.iloc[::, :look_back:].values
y_train = cesta.iloc[::, look_back::].values
X_val = cesta.iloc[-1::, -12::].values

#remodela os dados para formato (batches, sequencia de tempo, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

#Cria o modelo
model=Sequential()
model.add(Conv1D(filters=24, kernel_size=3, input_shape=(look_back,1), activation='tanh'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(15, activation='tanh'))
model.add(Dense(forecast_horizon))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

#Treina e prever (forecast_horizon) quantidade de tempo
history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
forecast = model.predict(X_val, batch_size=1)

resultados = [valor for valor in forecast[0]]
objeto = objeto.replace(" ",'_')
output = {objeto.lower():str(resultados)}
filename = f"previsao_CNN_{objeto.lower()}_{regiao.lower()}.json"
pasta_atual = os.getcwd()
full_url = Path(pasta_atual) / 'resultados' / 'previsoes_cesta' / filename
os.makedirs(os.path.dirname(full_url), exist_ok=True)
with open(full_url,"w") as file:
    json.dump(output,file,ensure_ascii=False)
print("Arquivo com resultados gerados")


Epoch 1/150
199/199 [==============================] - 0s 555us/step - loss: 0.0017
Epoch 2/150
199/199 [==============================] - 0s 556us/step - loss: 0.0014
Epoch 3/150
199/199 [==============================] - 0s 707us/step - loss: 0.0012
Epoch 4/150
199/199 [==============================] - 0s 566us/step - loss: 0.0011
Epoch 5/150
199/199 [==============================] - 0s 471us/step - loss: 9.9072e-04
Epoch 6/150
199/199 [==============================] - 0s 456us/step - loss: 9.4361e-04
Epoch 7/150
199/199 [==============================] - 0s 476us/step - loss: 9.1117e-04
Epoch 8/150
199/199 [==============================] - 0s 642us/step - loss: 8.8660e-04
Epoch 9/150
199/199 [==============================] - 0s 536us/step - loss: 8.6650e-04
Epoch 10/150
199/199 [==============================] - 0s 491us/step - loss: 8.4921e-04
Epoch 11/150
199/199 [==============================] - 0s 526us/step - loss: 8.3386e-04
Epoch 12/150
199/199 [========================

# Prever Produtos

In [3]:
path_inicial = r'C:\\Users\\mathe\\Documents\\NotebooksMateriais\\previsao_cestas\\datasets_produtos\\'

regioes = ['ilheus','itabuna']
produtos = ['acucar', 'arroz', 'banana', 'cafe', 'carne', 'farinha', 'feijao', 'leite', 
            'manteiga', 'oleo', 'pao', 'tomate']

for regiao in regioes:
    for objeto in produtos:
        #Carrega os dados
        path = f'{path_inicial}{regiao}/{objeto}_{regiao}.xlsx'
        cesta = pd.read_excel(path)
        cesta.drop(['ano'], axis=1, inplace=True)
        cesta = cesta / 1000
        cesta = cesta.iloc[::]

        #Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
        for n_step in range(1, look_back + forecast_horizon):
            cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
        cesta.dropna(inplace=True)
        cesta.reset_index(drop=True, inplace=True)

        #Divide os dados em Treino e Teste
        X_train = cesta.iloc[::, :look_back:].values
        y_train = cesta.iloc[::, look_back::].values
        X_val = cesta.iloc[-1::, -12::].values

        #remodela os dados para formato (batches, sequencia de tempo, features)
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

        #Cria o modelo
        model=Sequential()
        model.add(Conv1D(filters=24, kernel_size=3, input_shape=(look_back,1), activation='tanh'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(15, activation='tanh'))
        model.add(Dense(forecast_horizon))
        model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

        #Treina e prever (forecast_horizon) quantidade de tempo
        history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
        forecast = model.predict(X_val, batch_size=1)

        resultados = [valor for valor in forecast[0]]
        objeto = objeto.replace(" ", '_')
        output = {objeto.lower():str(resultados)}
        filename = f"previsao_CNN_{objeto.lower()}_{regiao.lower()}.json"
        pasta_atual = os.getcwd()
        full_url = Path(pasta_atual) / 'resultados' / 'previsoes_produtos' / f'{regiao}' /filename
        os.makedirs(os.path.dirname(full_url), exist_ok=True)
        with open(full_url,"w") as file:
            json.dump(output,file,ensure_ascii=False)
        print("Arquivo com resultados gerados")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\\\Users\\\\mathe\\\\Documents\\\\NotebooksMateriais\\\\previsao_cestas\\\\datasets_produtos\\\\ilheus/acucar_ilheus.xlsx'